In [3]:
# import tensorflow and numpy
import tensorflow as tf
import numpy as np

In [19]:
# Hyperparameters
# number of points for input data
vals = 4
# size of frame (1d in this case)
max_answer = 60

# number of gradients to learn, it goes up to
# the maximum linear line for this size of frame
gradients = max_answer/(vals)+1

iterations = 5000

# limit gpu memory usage
gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=0.5)
learning_rate = 1e-5

In [20]:
#  we create input data as numbers with extra for label
def training_data():
    
    M = 0
    rows = 1
    # X and Y values of equation y = Mx+b
    X = np.arange(vals+1)
    Y = np.array([])
    
    # append new line for each gradient and reshape
    for i in range(gradients):
        
        Y = np.append(Y, X*M).reshape(rows,vals+1)
            
        rows += 1
        M+=1

    return(Y.astype(np.int32),np.size(Y,0))

#print(training_data())

In [21]:
# y arrays and number of them form before
training_line_data,training_lines = training_data()

# set lengths to set when formatting input data
length = max_answer
full_length = length*vals

# function to format data
# variables are line data in number form and mode
def format_data(training_line_data, mode):
            
    # if its in training mode
    if mode == 'train':
        # set number of times to loop
        number_points = vals
        number_lines = training_lines
        # create labels and input data
        labels = np.zeros([training_lines,1,gradients])
        input_data = np.zeros([training_lines,vals,length])
        
    else:
        # set number of times to loop
        number_points = vals
        number_lines = 1
        # set first value to take away from the array to remove b
        first_value = test_line[0]
        training_line_data = np.subtract(training_line_data,first_value)
        # create input data
        input_data = np.zeros([vals,length])
        
    # loop over nummber of lines
    for i in range(number_lines):
        # loop over number of values to plot
        for a in range(number_points):
            
            # set the specified input data values to 1
            if mode == 'train':
                input_data[i,a,training_line_data[i,a]] = 1
            else:
                input_data[a,training_line_data[a]] = 1
            
        # set the specified label values to 1 if in train mode
        if mode == 'train':
            labels[i,0,training_line_data[i,1]] = 1
    
    # reshape and return data
    if mode == 'train':
        input_data = input_data.reshape(number_lines,1,full_length)
        return(input_data.astype(np.float32),labels)
    else:
        input_data = input_data.reshape(1,full_length)
        return(input_data.astype(np.float32))


#print(format_data(training_line_data, 'train'))

In [22]:
# define functions to create weights/biases
def weight_variable(shape):
  initial = tf.truncated_normal(shape, stddev=0.1)
  return tf.Variable(initial)

def bias_variable(shape):
  initial = tf.constant(0.1, shape=shape)
  return tf.Variable(initial)

In [23]:
def conv2d(x, W):
  return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

def max_pool_2x2(x):
  return tf.nn.max_pool(x, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

In [24]:
def deepnn(x):
    W_conv1 = weight_variable([vals, length, 1, 32])
    b_conv1 = bias_variable([32])

    x_image = tf.reshape(x, [-1,vals,length,1])

    h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)
    h_pool1 = max_pool_2x2(h_conv1)
    
    W_conv2 = weight_variable([vals, length, 32, 64])
    b_conv2 = bias_variable([64])

    h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
    h_pool2 = max_pool_2x2(h_conv2)

    W_fc1 = weight_variable([(vals/4) * (length/4) * 64, 1024])
    b_fc1 = bias_variable([1024])

    h_pool2_flat = tf.reshape(h_pool2, [-1, (vals/4)*(length/4)*64])
    h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)
    
    keep_prob = tf.placeholder(tf.float32)
    h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)
    
    W_fc2 = weight_variable([1024, gradients])
    b_fc2 = bias_variable([gradients])

    y_conv = tf.matmul(h_fc1_drop, W_fc2) + b_fc2
    
    return(y_conv, keep_prob)

In [25]:
# x placeholder for input data
x = tf.placeholder(tf.float32, [None, full_length])
# y_ placeholder for labels
y_ = tf.placeholder(tf.float32, [None, gradients])

# set y_conv and keep_prob by running the deepnn function inputting x
y_conv, keep_prob = deepnn(x)

# work out error with cross etntropy
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y_conv))

# set optimizer to  AdamOptimizer
train_step = tf.train.AdamOptimizer(learning_rate).minimize(cross_entropy)

# return boolean whether on whether the prediction(s) are equal to the label(s)
correct_prediction = tf.equal(tf.argmax(y_conv,1), tf.argmax(y_,1))

# turns boolean into float to represent accuracy percentage
accuracy = tf.reduce_mean(tf.cast(correct_prediction, dtype=tf.float32))

# run session
sess = tf.InteractiveSession(config=tf.ConfigProto(gpu_options=gpu_options))
   
# initialize variables
sess.run(tf.global_variables_initializer())

# training loop
for i in range(iterations):

    # select data for training and fro accuracy testing
    x_data, y_data = format_data(training_line_data, 'train')
    x_acc_test = x_data.reshape(training_lines,-1)
    y_acc_test = y_data.reshape(training_lines,-1)
    
    x_data = x_data[i%training_lines]
    y_data = y_data[i%training_lines]

    # print accuracy and error
    if i % (iterations/20) == 0:
        train_accuracy = sess.run(accuracy, feed_dict={x:x_acc_test, y_: y_acc_test, keep_prob: 1.0})
        error = sess.run(cross_entropy, feed_dict={x:x_data, y_: y_data, keep_prob: 1.0})
        
        print 'step', i, 'training accuracy', train_accuracy,  '\nerror =', error
        
    train_step.run(feed_dict={x: x_data, y_: y_data, keep_prob: 0.5})
    
x_data, y_data = format_data(training_line_data, 'train')

x_acc_test = x_data.reshape(training_lines,-1)
y_acc_test = y_data.reshape(training_lines,-1)

print("test accuracy %g" % sess.run(accuracy, feed_dict={x: x_acc_test, y_: y_acc_test, keep_prob: 1.0}))

step 0 training accuracy 0.0625 
error = 9.0246
[False]
step 250 training accuracy 0.1875 
error = 3.40585
[False]
step 500 training accuracy 0.625 
error = 1.87503
[False]
step 750 training accuracy 0.75 
error = 0.160423
[ True]
step 1000 training accuracy 0.9375 
error = 0.317299
[ True]
step 1250 training accuracy 1.0 
error = 1.09436
[ True]
step 1500 training accuracy 1.0 
error = 0.0128546
[ True]
step 1750 training accuracy 1.0 
error = 0.0159946
[ True]
step 2000 training accuracy 1.0 
error = 0.00774843
[ True]
step 2250 training accuracy 1.0 
error = 0.0160237
[ True]
step 2500 training accuracy 1.0 
error = 0.0095082
[ True]
step 2750 training accuracy 1.0 
error = 0.000773369
[ True]
step 3000 training accuracy 1.0 
error = 0.00582829
[ True]
step 3250 training accuracy 1.0 
error = 0.00278621
[ True]
step 3500 training accuracy 1.0 
error = 0.000464927
[ True]
step 3750 training accuracy 1.0 
error = 0.000838286
[ True]
step 4000 training accuracy 1.0 
error = 0.000557625

In [16]:
# initialize array to record test values
test_line = np.array([])

print 'input values, the max gradient is', gradients-1
# loop to ask user for input values
for i in range(vals):
    print '\nvalue',i+1,'?'
    val = int(input())
    test_line = np.append(test_line, val)
    
# set test line to int32 and setlast value
test_line = test_line.astype(np.int32)
last_val = test_line[-1]

# use format data function to set data to binary
input_array = format_data(test_line, 0)
#print input_array

input values, the max gradient is 160

value 1 ?
1

value 2 ?
12

value 3 ?
20

value 4 ?
31


In [17]:
probs = sess.run(y_conv, feed_dict={x:input_array, keep_prob:1.})
# now we have found the predicted gradient,
# we add the last value to get the next value
learnt_ans = np.argmax(probs)+last_val
# "true" answer, should work with all linear lines
# this numerically works out hte gradient by findingthe difference,
# then also adds the last value
answer = (test_line[1]-test_line[0])+last_val

if answer == learnt_ans:
    print 'Correct'
    
    # print learnt answer and it probability
    print '\nLearnt answer =', learnt_ans
    print 'Probaility of Gradient', np.argmax(probs), '=', probs[0,learnt_ans-last_val]
else:
    print 'Wrong'
    
    # print learnt answer and it probability
    print '\nLearnt answer =', learnt_ans
    print 'Probaility of Gradient', np.argmax(probs), '=', probs[0,learnt_ans-last_val]
    
    # print true answer and its probabilities
    print '\nNumerical answer =', answer
    print 'Probability of Gradient', test_line[1]-test_line[0], '=', probs[0,answer-last_val]
    

# print the probabilities
print '\n\n', probs

Wrong

Learnt answer = 95
Probaility of Gradient 64 = 6.08777

Numerical answer = 42
Probability of Gradient 11 = -3.21247


[[  1.10001612e+00   7.09558964e-01   2.73033142e+00   7.06519425e-01
   -1.21905768e+00   1.05500007e+00   2.50275910e-01  -7.24527717e-01
   -1.99253845e+00   1.95721650e+00  -3.92215788e-01  -3.21247482e+00
    1.61492020e-01  -2.08394337e+00   2.77687025e+00   1.38325822e+00
   -9.72138345e-03  -4.81650889e-01   3.30665350e+00  -1.56368852e+00
    7.48526752e-01   1.06613684e+00   2.75988430e-01   2.90247262e-01
    2.28166723e+00   2.32602191e+00   1.67041302e+00  -4.62210029e-01
   -2.90325975e+00  -2.62748384e+00  -1.69562733e+00   2.81702542e+00
    1.47609460e+00   2.11728406e+00   5.42709923e+00  -9.56273973e-01
   -1.72192252e+00   1.60939026e+00  -1.48714745e+00  -5.30712318e+00
   -2.78889513e+00  -1.96424496e+00   1.74495995e+00  -2.34901738e+00
    1.25798658e-01  -3.27890515e+00  -4.40883338e-01   2.20811319e+00
   -3.29399800e+00  -1.63021266e+00